In [ ]:
import numpy as np
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import matplotlib.style as style

style.use(
    "https://raw.githubusercontent.com/dominik-strutz/dotfiles/main/mystyle.mplstyle"
)

In [ ]:
from dased.criteria import RaySensitivity
from dased.optimisation import DASOptimizationProblem, DASArchipelago

In [ ]:
from cdv_setup import topo_data, design_space_full, shoulder_area

In [ ]:
cable_properties = {
    "elevation": topo_data,
    "cable_length": 2500,
    "spacing": 20,
    "fixed_points": np.array([[1550.0, 1400.0]]),
    "signal_decay": 0.4,
}

optimisation_properties = {
    "bounds": [
        [design_space_full.bounds[0], design_space_full.bounds[2]],
        [design_space_full.bounds[1], design_space_full.bounds[3]],
    ],
    "spatial_constraints": design_space_full.envelope.difference(design_space_full),
}

In [ ]:
##############################################################################
##############################################################################
############### THIS HAS TO BE FIRST PLOT IN THE NOTEBOOK
############### OTHERWISE IT WILL NOT WORK, FOR SOME REASON
##############################################################################
##############################################################################

from interactive_proposal import select_points

proposal_points = select_points(
    design_space=design_space_full,
    cable_properties=cable_properties,
    filename="data/benchmark/proposal_points_benchmark.pkl",
)

##############################################################################
############### IF THIS IS CALLED IT WILL NOT WORK AGAIN
############### BUT IT IS NEEDED TO HAVE LATE PLOTS DISPLAYED
##############################################################################

%matplotlib inline

In [ ]:
dx = dy = 20

n_x = int((design_space_full.bounds[2] - design_space_full.bounds[0]) / dx)
n_y = int((design_space_full.bounds[3] - design_space_full.bounds[1]) / dy)

print(f"n_x = {n_x}, n_y = {n_y}")

design_criterion = RaySensitivity(
    data_type="rayleigh",
    n_points=(n_x, n_y),
    reference_distance=1_000.0,
    x_range=[design_space_full.bounds[0], design_space_full.bounds[2]],
    y_range=[design_space_full.bounds[1], design_space_full.bounds[3]],
    roi=shoulder_area,
    # criterion='RER', criterion_kwargs=dict(threshold=1e-4)
    criterion="D",
    criterion_kwargs=dict(threshold=1e-4, penalty=10 * np.log(1e-4), normalize=True),
)

In [ ]:
# import os

# def rename_knot_to_anchor(root_dir):
#     """
#     Recursively rename all files and folders containing 'knot' to 'anchor'
#     """
#     # First pass: collect all items that need renaming
#     items_to_rename = []

#     for dirpath, dirnames, filenames in os.walk(root_dir, topdown=False):
#         # Collect files that need renaming
#         for filename in filenames:
#             if 'knot' in filename.lower():
#                 old_path = os.path.join(dirpath, filename)
#                 new_filename = filename.replace('knot', 'anchor').replace('Knot', 'Anchor').replace('KNOT', 'ANCHOR')
#                 new_path = os.path.join(dirpath, new_filename)
#                 items_to_rename.append(('file', old_path, new_path))

#         # Collect directories that need renaming
#         for dirname in dirnames:
#             if 'knot' in dirname.lower():
#                 old_path = os.path.join(dirpath, dirname)
#                 new_dirname = dirname.replace('knot', 'anchor').replace('Knot', 'Anchor').replace('KNOT', 'ANCHOR')
#                 new_path = os.path.join(dirpath, new_dirname)
#                 items_to_rename.append(('dir', old_path, new_path))

#     # Second pass: perform the renaming
#     renamed_count = 0
#     for item_type, old_path, new_path in items_to_rename:
#         try:
#             os.rename(old_path, new_path)
#             print(f"Renamed {item_type}: {old_path} -> {new_path}")
#             renamed_count += 1
#         except Exception as e:
#             print(f"Error renaming {old_path}: {e}")

#     print(f"\nTotal items renamed: {renamed_count}")

# # Run the renaming function
# rename_knot_to_anchor('data/benchmark')

In [ ]:
import os


def optimisation_wrapper(
    n_generations=500,
    population_size=128,
    migrate_every=20,
    N_knots=8,
    k=1,
    n_islands=10,
    random_seed=1,
    show_progress=True,
):
    filename_core = f"optimisation_{n_generations}gens_{population_size}pop_{migrate_every}migrates_{N_knots}anchors_{k}k"

    os.makedirs(f"data/benchmark/{filename_core}", exist_ok=True)

    udp = DASOptimizationProblem(
        design_criterion,
        verbose=0,
        k=k,
        N_knots=N_knots,
        **cable_properties,
        **optimisation_properties,
    )

    archipelago = DASArchipelago(
        udp,
        n_islands=n_islands,
        population_size=population_size,
        verbose=0,
        random_seed=random_seed,
    )

    # print(f"Loading initial population from file: data/benchmark/{filename_core}/initial_population_{filename_core}_{random_seed}rs")

    archipelago.initialize(
        proposal_points=proposal_points,
        perturb_proposal=50.0,
        perturb_knots=25.0,
        corr_len=500,
        corr_str=0.8,
        max_emergency_attempts=100_000,
        # min_length=1000,
        filename=f"data/benchmark/{filename_core}/initial_population_{filename_core}_{random_seed}rs",
        show_progress=show_progress,
    )

    # archipelago.get_file_info(f'data/benchmark/{filename_core}/optimisation_{filename_core}_{random_seed}rs')

    archipelago.optimize(
        n_generations=n_generations,
        migrate_every=migrate_every,
        show_progress=show_progress,
        filename=f"data/benchmark/{filename_core}/optimisation_{filename_core}_{random_seed}rs",
    )

    return archipelago


## Benchmark Population Number of Knots and Spline Degree

In [ ]:
n_generations = 500
population_size = 128
migrate_every = 20

In [ ]:
# anchor_list = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 25, 30, 40, 50]
anchor_list = [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 20, 25,]

k_list = [1, 3]

anchor_k_dict = {}
N_random = 10

for N_knots in tqdm(anchor_list, leave=True, desc="Running optimisations"):
    for k in k_list:
        # print(f"Running with N_knots={N_knots}, k={k}")
        for rseed in range(N_random):
            anchor_k_dict[(N_knots, k, rseed)] = optimisation_wrapper(
                n_generations=n_generations,
                population_size=population_size,
                migrate_every=migrate_every,
                N_knots=N_knots,
                k=k,
                random_seed=rseed,
            )
            # print(f"Running with N_knots={N_knots}, k={k}, rseed={rseed}: {anchor_k_dict[(N_knots, k, rseed)].get_best_single()[1]:.4f}")


In [ ]:
def get_mean_std_fitness(pop_mig_dict, *args):
    fitness_histories = []
    for rseed in range(N_random):
        fitness_history = pop_mig_dict[(*args, rseed)].get_fitness_history()
        best_fitness = np.max(fitness_history, axis=1)
        fitness_histories.append(np.array(best_fitness, dtype=float))

    fitness_histories = np.stack(fitness_histories, axis=0)

    mean_fitness = np.mean(fitness_histories, axis=0)
    std_fitness = np.std(fitness_histories, axis=0)

    return mean_fitness, std_fitness


In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
fig.set_facecolor("white")

colors = ["tab:blue", "tab:orange"]  # One color for each k in k_list

all_best_fitness = []
for N_knots in anchor_list:
    for k in k_list:
        for rseed in range(N_random):
            fitness_history = anchor_k_dict[(N_knots, k, rseed)].get_fitness_history()
            best_fitness = np.max(fitness_history, axis=1)
            all_best_fitness.append(best_fitness[-1])

ymin = np.floor(min(all_best_fitness) * 20) / 20
ymax = np.ceil(max(all_best_fitness) * 20) / 20
grid_lines = np.arange(ymin, ymax + 0.01, 0.1)

for g in grid_lines:
    ax.axhline(g, color="lightgray", linestyle="-", linewidth=0.7, zorder=-1)

width = 0.4
offsets = np.linspace(-width / 2, width / 2, len(k_list))
for idx, k in enumerate(k_list):
    data = []
    for N_knots in anchor_list:
        fitness_histories = []
        for rseed in range(N_random):
            fitness_history = anchor_k_dict[(N_knots, k, rseed)].get_fitness_history()
            best_fitness = np.max(fitness_history, axis=1)
            fitness_histories.append(best_fitness[-1])
        data.append(fitness_histories)
    positions = np.arange(len(anchor_list)) + offsets[idx]
    parts = ax.violinplot(
        data,
        positions=positions,
        showmeans=True,
        widths=width,
        showextrema=True,
    )
    for pc in parts["bodies"]:
        pc.set_facecolor(colors[idx])
        pc.set_edgecolor("none")
        pc.set_alpha(0.7)
        pc.set_zorder(2)
    for key in ["cmeans", "cmins", "cmaxes", "cbars"]:
        if key in parts:
            parts[key].set_color(colors[idx])
            parts[key].set_linewidth(1)
            parts[key].set_zorder(2)
    ax.plot([], [], color=colors[idx], label=f"k = {k}", linewidth=6)

# Add vertical lines between sets of violins for each N_knots
for i in range(1, len(anchor_list)):
    ax.axvline((i - 0.5), color="k", linestyle="-", linewidth=1, zorder=0)
ax.set_xticks(range(len(anchor_list)))
ax.set_xticklabels(anchor_list)
ax.tick_params(axis="x", which="both", length=0)  # Hide x-ticks but show labels
ax.set_xlim(-0.5, len(anchor_list) - 0.5)

ax.xaxis.set_minor_locator(plt.NullLocator())  # Remove x minor ticks
ax.set_ylim(ymin, ymax)
ax.set_yticks(grid_lines)
ax.set_xlabel(r"$N_\mathrm{knots}$", fontsize=11)
ax.set_ylabel("Rayleigh D-Optimilality", fontsize=11)
              
ax.legend(
    title="Spline Degree",
    fontsize=10,
    title_fontsize=11,
    facecolor="white",
    framealpha=1,
    loc="lower right",
)

fig.savefig("figures/fitness_vs_N_knots.png", dpi=300, bbox_inches="tight")
fig.savefig("figures/fitness_vs_N_knots.pdf", dpi=300, bbox_inches="tight")

plt.show()


## Benchmark Population Size and Migration Frequency

In [ ]:
n_generations = 500
N_knots = 10
k = 1

In [ ]:
pop_size_list = [16, 32, 64, 128, 256]
# pop_size_list = [16, 32, 64, 128]

migrate_every_list = [1, 5, 10, 20, 50, 100, 500]
# migrate_every_list = [1, 5, 10, 50,]

N_random = 10

pop_mig_dict = {}

for pop_size in tqdm(pop_size_list, leave=True, desc="Running optimisations"):
    for migrate_every in migrate_every_list:
        # print(f"Running with pop_size={pop_size}, migrate_every={migrate_every}")
        for rseed in range(N_random):
            pop_mig_dict[(pop_size, migrate_every, rseed)] = optimisation_wrapper(
                n_generations=n_generations,
                population_size=pop_size,
                migrate_every=migrate_every,
                N_knots=N_knots,
                k=k,
                random_seed=rseed,
                show_progress=True,
            )
            # print(f"Running with pop_size={pop_size}, migrate_every={migrate_every}, rseed={rseed}: {pop_mig_dict[(pop_size, migrate_every, rseed)].get_best_single()[1]:.2f}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
fig.set_facecolor("white")

colors = plt.cm.viridis(np.linspace(0, 1, len(migrate_every_list)))

all_best_fitness = []
for pop_size in pop_size_list:
    for migrate_every in migrate_every_list:
        for rseed in range(N_random):
            fitness_history = pop_mig_dict[
                (pop_size, migrate_every, rseed)
            ].get_fitness_history()
            best_fitness = np.max(fitness_history, axis=1)
            all_best_fitness.append(best_fitness[-1])

ymin = np.floor(min(all_best_fitness) *20) /20
ymax = np.ceil(max(all_best_fitness) *20) /20
grid_lines = np.arange(ymin, 0.92, 0.05)

for g in grid_lines:
    ax.axhline(g, color="lightgray", linestyle="--", linewidth=0.7, zorder=-1)

width = 0.8 / len(migrate_every_list)  # Make sure all violins fit within each sector
sector_centers = np.arange(len(pop_size_list))
for idx, migrate_every in enumerate(migrate_every_list):
    data = []
    for pop_size in pop_size_list:
        fitness_histories = []
        for rseed in range(N_random):
            fitness_history = pop_mig_dict[
                (pop_size, migrate_every, rseed)
            ].get_fitness_history()
            best_fitness = np.max(fitness_history, axis=1)
            fitness_histories.append(best_fitness[-1])
        data.append(fitness_histories)
    # Evenly distribute violins within each sector
    positions = sector_centers + (idx - (len(migrate_every_list) - 1) / 2) * width
    parts = ax.violinplot(
        data, positions=positions, showmeans=True, widths=width, showextrema=True
    )
    for pc in parts["bodies"]:
        pc.set_facecolor(colors[idx])
        pc.set_edgecolor("none")
        pc.set_alpha(0.7)
        pc.set_zorder(2)
    for key in ["cmeans", "cmins", "cmaxes", "cbars"]:
        if key in parts:
            parts[key].set_color(colors[idx])
            parts[key].set_linewidth(1)
            parts[key].set_zorder(2)
    ax.plot(
        [],
        [],
        color=colors[idx],
        label=rf"$N_\text{{migrate}}={migrate_every}$"
        if migrate_every != 500
        else r"$\text{no migration}$",
        linewidth=6,
    )

# Add vertical lines between sets of violins for each pop_size
for i in range(1, len(pop_size_list)):
    ax.axvline(i - 0.5, color="k", linestyle="-", linewidth=1, zorder=0)

ax.set_xticks(range(len(pop_size_list)))
ax.set_xticklabels(pop_size_list)
ax.set_xlim(-0.5, len(pop_size_list) - 0.5)

ax.tick_params(axis="x", which="both", length=0)  # Hide x-ticks but show labels
ax.xaxis.set_minor_locator(plt.NullLocator())  # Remove x minor ticks

ax.set_ylim(ymin, 0.92)
ax.set_yticks(grid_lines)
ax.set_xlabel("Population Size", fontsize=11)
ax.set_ylabel("Rayleigh D-Optimilality", fontsize=11)
ax.legend(fontsize=8, facecolor="white", framealpha=1, loc="lower right", ncol=1)

fig.savefig("figures/fitness_vs_pop_size.png", dpi=300, bbox_inches="tight")
fig.savefig("figures/fitness_vs_pop_size.pdf", dpi=300, bbox_inches="tight")

plt.show()
